In [1]:
"""
Data Preprocessing
"""

import os
import re
import pandas as pd

# read train data
file_dir = '/Users/dnhb/PycharmProjects/SML_Ass1'
file_name = "train.txt"

def parse(data:list)->list: 
    # Python Notation: https://www.python.org/dev/peps/pep-3107/
    """
    Prase each line of the data
    Return a nested list, such as [[id1, id2, id2,...],...]
    """
    parsed_data = []
    for line in data:
        # remove \n at the end of each line
        line = re.sub(r"(?<=\d)\n", "", line)
        # split IDs by \t
        pattern = re.compile("(?<=\d)\t(?=\d)")
        line = re.split(pattern, line)
        parsed_data.append(line)
    return parsed_data

with open(os.path.join(file_dir, file_name)) as f:
    train_set = f.readlines()
    train_set = parse(train_set)
    
# read test data
test_name = "test-public.txt"
with open(os.path.join(file_dir, test_name)) as f:
    test_set = f.readlines()
    test_set = parse(test_set)
    test_set.pop(0)

# turn ID into int
for i in range(0, len(test_set)):
    test_set[i][0] = int(test_set[i][0])

In [48]:
"""
Sigma Data For Similarity
"""

from collections import defaultdict

train_dic = {}
for item in train_set:
    train_dic[item[0]] = item[1:]

# construct SigmaOut dic for each node
SigmaOut = defaultdict(set)

def getSigmaOut(node:str, graph:dict) -> set:
    SigmaOut = set()
    try:
        for item in graph[node]:
            SigmaOut.add(item)
        return SigmaOut
    except:
        return SigmaOut

for kw in train_dic:
    SigmaOut[kw] = getSigmaOut(kw, train_dic)

# construct SigmaIn dic for each node
SigmaIn = defaultdict(set)

for kw in train_dic:
    for followee in train_dic[kw]:
        if SigmaIn[followee]:
            SigmaIn[followee].add(kw)
        else:
            SigmaIn[followee] = {kw}

def getSigma(node:str) -> set:
    return SigmaIn[node].union(SigmaOut[node])

In [49]:
"""
Similarity Functions
"""

import math

    
def adar1(a:str, b:str) -> float:
    score = 0
    SigmaA = getSigma(a)
    SigmaB = getSigma(b) 
    SigmaV = SigmaA.intersection(SigmaB)
    if SigmaV:
        for v in SigmaV:
            if getSigma(v) and len(getSigma(v)) != 1:
                   score += (1/math.log(len(getSigma(v))))
            else:
                   pass
        return score
    else:
        return 0

def adar2(a:str, b:str) -> float:
    score = 0
    SigmaOutA = SigmaOut[a]
    if SigmaOutA:
        for node in SigmaOutA:
            score += adar1(b, node)
        return float(score/len(SigmaOutA))
    else:
        return 0
    
def adar3(a:str, b:str) -> float:
    score = 0
    SigmaInB = SigmaIn[b]
    if SigmaInB:
        for node in SigmaInB:
            score += adar1(a, node)
        return float(score/len(SigmaInB))
    else:
        return 0
  
def jaccard1(a:str, b:str) -> float:
    SigmaA = getSigma(a)
    SigmaB = getSigma(b)
    return float(len(SigmaA.intersection(SigmaB)) / len(SigmaA.union(SigmaB)))

def jaccard2(a:str, b:str) -> float:
    score = 0
    SigmaOutA = SigmaOut[a]
    if SigmaOutA:
        for node in SigmaOutA:
            score += jaccard1(b, node)
        return float(score/len(SigmaOutA))
    else:
        return 0

def jaccard3(a:str, b:str) -> float:
    score = 0
    SigmaInB = SigmaIn[b]
    if SigmaInB:
        for node in SigmaInB:
            score += jaccard1(a, node)
        return float(score/len(SigmaInB))
    else:
        return 0
    
def cosine1(a:str, b:str) -> float:
    SigmaA = getSigma(a)
    SigmaB = getSigma(b)
    return float(len(SigmaA.intersection(SigmaB)) / (len(SigmaA)*len(SigmaB)))


def cosine2(a:str, b:str) -> float:
    score = 0
    SigmaOutA = SigmaOut[a]
    if SigmaOutA:
        for node in SigmaOutA:
            score += cosine1(b, node)
        return float(score/len(SigmaOutA))
    else:
        return 0

def cosine3(a:str, b:str) -> float:
    score = 0
    SigmaInB = SigmaIn[b]
    if SigmaInB:
        for node in SigmaInB:
            score += cosine1(a, node)
        return float(score/len(SigmaInB))
    else:
        return 0

def common_neighbors1(a: str, b: str) -> float:
    SigmaA = getSigma(a)
    SigmaB = getSigma(b)
    return len(SigmaA.intersection(SigmaB))

def common_neighbors2(a: str, b: str) -> float:
     score = 0
     SigmaOutA = SigmaOut[a]
     if SigmaOutA:
         for node in SigmaOutA:
             score += common_neighbors1(b, node)
         return float(score / len(SigmaOutA))
     else:
         return 0

def common_neighbors3(a: str, b: str) -> float:
     score = 0
     SigmaInB = SigmaIn[b]
     if SigmaInB:
         for node in SigmaInB:
             score += common_neighbors1(a, node)
         return float(score / len(SigmaInB))
     else:
         return 0

def knn1(a:str, b:str) -> float:
    """
    knn1 = $w_{ab} = w_a^out$
    """
    return 1/math.sqrt(1+len(SigmaOut[a]))

def knn2(a:str, b:str) -> float:
    """
    knn2 = $w_{ab} = w_b^in$
    """
    return 1/math.sqrt(1+len(SigmaIn[b]))

def knn3(a:str, b:str) -> float:
    """
    knn3 = $w_{ab} = w_a^out  w_b^in$
    """
    return 1/math.sqrt(1+len(SigmaOut[a])) + 1/math.sqrt(1+len(SigmaIn[b]))

def knn4(a:str, b:str) -> float:
    """
    knn4 = $w_{ab} = w_a^out * w_b^in$
    """
    wAOut = 1/math.sqrt(1+len(SigmaOut[a]))
    wBIn = 1/math.sqrt(1+len(SigmaIn[b]))
    return  wAOut* wBIn

In [5]:
"""
Construct Train Data
Output:
1. train data: train_link
2. label of train data: train_label
"""

import random

train_dic = {}
for item in train_set:
    train_dic[item[0]] = item[1:]

# collect user list in the original train set
user_list = list(train_dic.keys())
random.seed(0)
# select 200 user randomly
user_train = random.sample(user_list, 200)

# construct positive dataset
    # the element of positive_list = the link that comes from traversing the adjacent list of one user
positive_set = set()
sink_set = set()
for user in user_train:
    for sink in train_dic[user]:
        positive_set.add((user, sink))
        sink_set.add(sink)
print("The length of positive list is", len(positive_set), ".\n")
print("The length of sink set is", len(sink_set), ".\n")

# construct negative dataset
    # the element (user1 -> user2) of negative_list = 
    # user1 = an element in the user list
    # user2 = any element except user1's adjacency
negative_set = set()

count=0
for user in user_train:
    for sink in sink_set:
        if (user, sink) not in positive_set:
            negative_set.add((user, sink))
    count=count+1
    if count%20==0:
        print(count)
print("The length of negative list is", len(negative_set), ".")

# construct train dataset
random.seed(0)
positive_train = random.sample(list(positive_set), 10000)
random.seed(0)
negative_train = random.sample(list(negative_set), 3000)
# create label for positive and negative train data
positive_label = [1] * len(positive_train)
negative_label = [0] * len(negative_train)
# combine positive and negative train data & label
#train_link = []
#train_link.extend(positive_train)
#train_link.extend(negative_train)
Y_train = []
Y_train.extend(positive_label)
Y_train.extend(negative_label)

del user_list, user_train
del positive_set, sink_set, negative_set, count
#del positive_train, negative_train, positive_label, negative_label

The length of positive list is 171315 .

The length of sink set is 144184 .

20
40
60
80
100
120
140
160
180
200
The length of negative list is 28665485 .


In [42]:
"""
Get Feature
"""

def getFeature(pair:tuple) -> float:
    node1 = pair[0]
    node2 = pair[1]
    result = []
    result.append(adar1(node1,node2))
    #result.append(salton(node1,node2))
    #result.append(allocation(node1,node2))
    #result.append(jaccard1(node1,node2))
    result.append(jaccard2(node1,node2))
    result.append(jaccard3(node1,node2))
    #result.append(cosine1(node1,node2))
    result.append(cosine2(node1,node2))
    result.append(cosine3(node1,node2))
    #result.append(knn1(node1,node2))
    #result.append(knn2(node1,node2))
    #result.append(knn3(node1,node2))
    #result.append(knn4(node1,node2))
    #result.append(AA(node1,node2))
    #result.append(CNPA(node1,node2))
    print(result)
    return result

In [6]:
import multiprocessing as mp
import time
    
if __name__ == '__main__':
         
    # construct the # of pools corresponding to the cpu_count in ur PC
    pool = mp.Pool(mp.cpu_count())
    
    startTime = time.time()
    
    positive = pool.map(getFeature, positive_train)
    pool.close()
    pool.join()
    
    endTime = time.time()
    print("Total time:" + (endTime - startTime).__str__())
    
    pool = mp.Pool(mp.cpu_count())
    
    startTime = time.time()
    
    negative = pool.map(getFeature, negative_train)
    pool.close()
    pool.join()
    
    endTime = time.time()
    print("Total time:" + (endTime - startTime).__str__())
    
    X_train = []
    X_train.extend(positive)
    X_train.extend(negative)

[0.5141733749250673, 0.009478672985781991, 0.12021559063659151, 0.17176520116583846, 0.0013637913399249914, 0.03806672442777461, 0.00042176553261524753]
[0.10032186572477783, 7.807620237351655e-05, 0.0591198268150176, 0.18747524995280837, 1.3017782290608973e-05, 0.036665495054905496, 1.6716598399143588e-05]
[2.6082497749396465, 0.023131672597864767, 0.011871606558109305, 0.0368903444114964, 0.0001574497977375675, 0.0004230720516740337, 0.0002158525760336211]
[6.799319808550232, 0.02861230329041488, 0.06455786691132111, 0.045737497359451124, 0.0002069879119059447, 0.0011485494319208936, 4.030146100743335e-05]
[4.929727399328508, 0.014338112704234745, 0.04528063288443973, 0.04664285017258778, 0.0001730577288386619, 0.00214684563508597, 2.8046598977116423e-05]
[0.7876853945377766, 0.0022579734688117415, 0.0401155739228647, 0.10565567044102454, 0.00020544427324088342, 0.010269298241217912, 3.111332619887418e-05]
[0, 0.0, 0.19342717622254627, 0.5105654681688319, 0.0, 0.1598911561547628, 5.9

[0.35065474194929597, 0.00032202662086732505, 0.022032587216838866, 0.1330223141216554, 2.479789713832267e-05, 0.0036647660394006068, 2.3054939621102936e-05]
[0.11519169974880678, 0.00011435105774728416, 0.12034462879872636, 0.26014205841147564, 2.8597574925646306e-05, 0.08106259091700009, 3.2531825678906144e-05]
[0, 0.0, 0.2164313066244827, 1.0, 0.0, 0.2164313066244827, 7.810669374365383e-05]
[4.70834828912932, 0.0462046204620462, 0.052308229398185295, 0.032031733452756485, 0.0002893359052080463, 0.0009424969748638113, 5.866548559095666e-05]
[0.11907655034710998, 5.8014735742878694e-05, 0.18774737877352965, 0.5172792362768497, 2.9009050823857045e-05, 0.15249869254154413, 3.374148723586076e-05]
[6.699932758324128, 0.007290786136939983, 0.07912360508687663, 0.03814081945409956, 3.266330944651312e-05, 0.001117399327724216, 9.865591645178407e-06]
[9.144429722786692, 0.008685395668024877, 0.14341403946266043, 0.16381622017316905, 8.617883064902234e-05, 0.0023466298268830475, 4.092299168748

[0, 0.0, 0.31948133166673337, 1.0, 0.0, 0.31948133166673337, 0.00011439029970258523]
[0.43287476076138043, 9.970835306727821e-05, 0.07531631554914094, 0.09150957820297925, 7.671594387461545e-06, 0.023075142035346833, 4.113531385183747e-06]
[0.4692515181489912, 0.000290056851142824, 0.08165292174765666, 0.17426205791699373, 4.144150117693863e-05, 0.04459414285023913, 1.2595383503469353e-05]
[5.208595725166875, 0.005776018825542839, 0.10421195732288151, 0.07336092901166083, 5.9821731240902115e-05, 0.0022701929957589313, 1.2272803123504002e-05]
[0.2748779375498046, 0.003942181340341655, 0.038909347554160575, 0.2570913964519289, 0.000986842105263158, 0.014970476032332238, 0.00033841467182376295]
[0, 0.0, 0.12489250571587678, 1.0, 0.0, 0.12489250571587678, 0.0064516129032258064]
[0.0865405559716985, 5.8011370228564796e-05, 0.139519001464429, 0.35244849293833563, 1.9339367215904696e-05, 0.10146331171932993, 2.1395802977272874e-05]
[6.05700743473171, 0.0036985668053629217, 0.06153810521119994

[0.5623768560777629, 0.006437768240343348, 0.029572730245513715, 0.046503759716579704, 0.0002628696604600219, 0.00467449866628122, 5.5346235209272606e-05]
[0.13186077721231096, 0.0001142334932602239, 0.05699950628147576, 0.08238273105288307, 8.799253823275786e-06, 0.026414924536595715, 1.3026312001024702e-05]
[1.212833412900555, 0.0002490597992578018, 0.02297833133352133, 0.027575718197189035, 4.704279577216986e-06, 0.005101297453326512, 2.95809848383455e-06]
[7.284838109655248, 0.006612995974698103, 0.002926253813718497, 0.00928810329428538, 3.7562083591640704e-06, 0.00011330689490630405, 4.142922892963996e-06]
[1.0717867626024093, 0.013223140495867768, 0.06466664928626403, 0.07610733393941993, 0.0006152899553914783, 0.008895088529286499, 0.0001660860793706757]
[0.07383035914027619, 0.000275027502750275, 0.053113265122451105, 0.03730997111513233, 8.408166010829718e-06, 0.009375756908191948, 1.1376629929003282e-05]
[14.540504433654538, 0.048104956268221574, 0.05898969587985424, 0.02442

[0.7644117019267122, 0.00019938688532761758, 0.038667354441610374, 0.06913841292689575, 8.672237133652183e-06, 0.010727209942892458, 2.9387763330686086e-06]
[0.6986743880358263, 0.0008071025020177562, 0.017223410031676732, 0.018686408317923956, 9.581494028676043e-06, 0.00798096731489627, 3.965063942438371e-06]
[0.8628668811759129, 0.0004679457182966776, 0.02190982987366566, 0.045247340994196576, 1.874560649847692e-05, 0.008767704785980322, 9.07582080543338e-06]
[5.694879894407958, 0.14814814814814814, 0.09218186936320912, 0.23121839972335168, 0.002663622526636225, 0.0027223337780501364, 0.0023866314242209003]
[0.6776844698003457, 0.0004060795915999536, 0.08421944843028094, 0.1578764316009193, 4.512519017044429e-05, 0.03971493616013066, 1.1074891598859409e-05]
[14.391800131108223, 0.015161221439248345, 0.17495277846719254, 0.09745680960499417, 7.553946884050107e-05, 0.0021957411461334666, 1.9459037559332517e-05]
[0, 0.0, 0.07163244734811272, 0.21133972207574941, 0.0, 0.03426166815100428

...

[0, 0.0, 0.0011889173341615223, 0.0005826732031364176, 0.0, 1.594253655918075e-05, 3.855555770304918e-06]
[0, 0.0, 0.0004327731092436975, 0.00025755500920033236, 0.0, 9.09090909090909e-05, 3.4027446474780166e-06]
[0, 0.0, 0.0021404158270619386, 0.00025926436035562955, 0.0, 0.0005634281509846457, 2.6786157743215544e-06]
[0.09434557625106478, 0.00392156862745098, 0.005208909300311684, 0.00043104250438788435, 0.00034153005464480874, 0.0005681958792336558, 1.773655430296985e-06]
[0, 0.0, 0.0008965852553292003, 0.007027445772465693, 0.0, 0.0008965852553292003, 7.65140073651058e-06]
[0, 0.0, 0.00019172214309647517, 0.014180374361883154, 0.0, 0.00019172214309647517, 2.735163923844284e-05]
[0.10400266106678616, 0.0015923566878980893, 0.009972937247085832, 9.429912587866783e-05, 7.48839299086416e-05, 0.0003640068308231249, 4.324137316457676e-06]
[0.3903702587932092, 0.00042941492216854533, 0.01639856571349251, 0.05411898298404612, 3.306386285109689e-05, 0.0014772027767138765, 9.343443504045104e

[0, 0.0, 0.00042720482600197275, 0.009014906504857126, 0.0, 0.00010936676086300098, 1.2905068816552325e-05]
[0, 0.0, 0.0032986450278685185, 0.0009331140081473771, 0.0, 0.0002005207067016998, 2.859247515678286e-06]
[0.14531902772192534, 0.0017825311942959, 0.002850467171873295, 0.0017744069089372328, 3.5861574323112784e-05, 1.6229085958664015e-05, 3.532450307513607e-05]
[0, 0.0, 0.0009303239219317351, 0.00016980140539805887, 0.0, 4.458555364333785e-05, 1.938640980470198e-06]
[0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.40620306785376575, 0.006265664160401002, 0.01345556678945381, 0.005133538480653454, 0.0002475002475002475, 0.0011431160494917015, 1.0831624353423906e-05]
[0.874300820248511, 0.00700770847932726, 0.032553172430958247, 0.003804062925853538, 0.00016315342948508777, 0.0017236285136224179, 3.0686864200051536e-06]
[0.8218491295988004, 0.009846827133479213, 0.03944936905277048, 0.0040515042171522676, 0.00016137708445400752, 0.0010694589068848258, 5.810602596121487e-06]
[0, 0.0, 0.004040

[0, 0.0, 0.01909451795077771, 0.0022320137962768844, 0.0, 0.003100788248629842, 8.162748158400138e-07]
[0, 0.0, 0.004017489890533557, 0.0005175995451136658, 0.0, 0.00032672946904070194, 4.522124543573071e-06]
[0.16010896647730544, 0.0045045045045045045, 0.008236956124488312, 0.0012847863590371865, 0.00017678776628657296, 0.001136581681141386, 3.70219217206525e-06]
[0, 0.0, 5.4814733437975434e-05, 0.00012921957701686937, 0.0, 3.1789787000874902e-06, 9.571630303562289e-07]
[0.08358176452866961, 0.004464285714285714, 0.00034456845937189465, 0.0007864754043113194, 0.00019327406262079628, 1.5284553830611347e-05, 5.999440325983881e-06]
[0, 0.0, 0.005150471008645904, 0.003152614995028592, 0.0, 0.0003284831706179742, 2.0901220916048654e-05]
[0, 0.0, 0.02040816326530612, 0.0003818610147245861, 0.0, 0.005434782608695652, 2.57671137818349e-06]
[0, 0.0, 0.0043797915279387345, 0.002875710436252597, 0.0, 0.00021698562770987968, 1.9407309513503324e-05]
[0, 0.0, 0.0024977442039711794, 0.00059522333275

[0, 0.0, 0.00246570871550953, 0.00045156669329486843, 0.0, 0.0005129124606379316, 6.013130171678677e-06]
[0, 0.0, 0.00110712483139941, 0.0012882152097383598, 0.0, 1.5223622013473026e-05, 2.4710105048732122e-05]
[0, 0.0, 1.157374205011266e-06, 7.655254893297982e-05, 0.0, 5.787129576755333e-07, 1.183881323590292e-06]
[0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0.0, 0, 1.6603520177533504e-05, 0.0, 0, 5.535572092600862e-06]
[0.26285264099704897, 0.004559270516717325, 0.016212120211504574, 0.0016002704273673808, 0.00011539349180706209, 0.001207505676781987, 3.4909134364121878e-06]
[0, 0.0, 0.0038269169724290336, 1.805794176344029e-05, 0.0, 6.0226950283854166e-05, 2.0086823147245778e-06]
[0.1823792172481698, 0.005917159763313609, 0.007506948783361664, 0.0004885079727557912, 6.982265046781176e-05, 0.0001711227010421921, 3.4095506276021215e-06]
[0.07383035914027619, 0.0011904761904761906, 0.008579870591840654, 0.0006552667662022821, 2.9799153704034804e-05, 0.001153852298239342, 1.350404268195134e-06

[0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0.0, 0.00571211334343859, 0.0002363790119429028, 0.0, 0.00011661535460399818, 5.590010495846984e-06]
[0, 0.0, 0.004348117263075471, 0.003393528115298903, 0.0, 0.0007302708923990923, 1.5828635965824583e-05]
[0, 0.0, 6.12890740774956e-05, 0.003863477018394996, 0.0, 5.15183673438795e-06, 2.648989915284126e-05]
[0.09086603972658462, 0.00031308703819661864, 0.0015772535173519996, 0.001135720727690638, 6.620061434170109e-06, 0.00010255565670430833, 9.88650188568305e-07]
[0, 0.0, 0.020030665121578053, 1.2031504900907791e-05, 0.0, 0.0003614712114665383, 5.73848608158357e-07]
[0, 0.0, 0.002331146175895577, 0.0001498299812751353, 0.0, 7.132650463679728e-05, 4.886658213759288e-07]
[0, 0.0, 0.0041458656964202464, 0.000904144346142035, 0.0, 0.0002589057871558829, 3.99939567508595e-06]
[0, 0.0, 7.66389233764044e-06, 0.00011371389583807141, 0.0, 7.66389233764044e-06, 5.817335660267598e-07]
[0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.07383035914027619, 0.0008992805755395

In [8]:
title = ["adar1", "jaccard1", "jaccard2", "jaccard3", 
        "cosine1", "cosine2", "cosine3"]
test_pd = pd.DataFrame(columns = title, data = X_train)
test_pd.to_csv('/Users/dnhb/PycharmProjects/SML_Ass1/X_trainScore.csv',encoding='utf-8')

In [40]:
assert(len(X_train) == len(Y_train))

In [39]:
"""
This cell only works when reading data from local 
"""

# read X_train data
score_name = "X_trainScore.csv"
with open(os.path.join(file_dir, score_name)) as f:
    X_train = f.readlines()
    X_train = parse(X_train)
    X_train.pop(0)
    
for i in range(0, len(X_train)):
    X_train[i] = X_train[i][0].split(",")
    for j in range(0, 7):
        X_train[i][j] = float(X_train[i][j])

# construct Y
positive_label = [1] * 10000
negative_label = [0] * 3000
Y_train = []
Y_train.extend(positive_label)
Y_train.extend(negative_label)

"""
Step:
1. run this cell
2. run the *assert block*

if need to select feature, then
1. run the X_trian1 cell, select the features
2. edit and run *getFeature* cell
3. run the paparmeters of RF
4. gridsearch and so on 
"""

In [41]:
# "adar1",  "jaccard2", "jaccard3", "cosine2", "cosine3"
X_train1 = []
for item in X_train:
    new = []
    new.append(item[0])
    new.extend(item[2:4])
    new.extend(item[5:])
    X_train1.append(new)

In [43]:
from sklearn.ensemble import RandomForestRegressor
#from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import numpy as np

# Number of trees in random forest
n_estimators = [50, 100, 200]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [20,50]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 3, 5, 8, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 6, 8]
# Method of selecting samples for training each tree
#bootstrap = [True]
# Create the random grid
para_grid = {'n_estimators': n_estimators,
             #'max_features': max_features,
             #'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [44]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np


rf = RandomForestRegressor(bootstrap=True, criterion='mse',
           max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_weight_fraction_leaf=0.0, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False )
grid_search = GridSearchCV(estimator = rf, param_grid = para_grid, cv = 3, verbose=2, n_jobs = -1)
# Fit the random search model
grid_search.fit(X_train1, Y_train)
best_grid = grid_search.best_estimator_
print (best_grid)
print ("train success!")

Fitting 3 folds for each of 75 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   40.8s
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:  1.1min finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=50, n_jobs=1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
train success!


In [45]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

rf = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=50, n_jobs=1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
rf.fit(X_train1, Y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=50, n_jobs=1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
import multiprocessing as mp
import time

if __name__ == '__main__':
    
    def getPro(record:list) -> list:
        feature = getFeature((record[1], record[-1]))
        prediction = rf.predict([feature])
        pro = [record[0], prediction[0]]
        print(pro)
        return pro
    
    # construct the # of pools corresponding to the cpu_count in ur PC
    pool = mp.Pool(mp.cpu_count())
    
    startTime = time.time()
    
    result = pool.map(getPro, test_set)
    pool.close()
    pool.join()
    
    endTime = time.time()
    print("Total time:" + (endTime - startTime).__str__())
    
    title = ["Id", "Predicted"]
    test_pd = pd.DataFrame(columns = title, data = result)
    test_pd.to_csv('/Users/dnhb/PycharmProjects/SML_Ass1/RF2Score.csv',encoding='utf-8')

[0.21933342447606208, 0.0010560980236248873, 0.005766530849023141, 0.0002158649279589648, 1.5493731237062643e-05]
[316, 0.06]
[0, 0.0006682103297357281, 1.8060941139843826e-05, 0.00034322807471536454, 7.228122423328354e-08]
[442, 0.0]
[0, 0.004756335876775657, 0.0005961701829318619, 0.0011086098198699584, 3.06738185991771e-05]
[1, 0.0]
[0, 0.0004547391940738875, 0.007885478256716273, 5.10988236771775e-05, 0.00022864230811537524]
[2, 0.06]
[0.4627293341056383, 0.006910701545848949, 0.024294678294895098, 0.0005541850751437438, 0.000186636503156485]
[3, 0.74]
[0.822336943717954, 0.012449053139136008, 0.006981832564143511, 0.0005297209380207123, 3.751707002281268e-05]
[127, 0.48]
[0, 0.0, 5.02373715807189e-05, 0.0, 5.58585883976126e-07]
[128, 0.0]
[1.0980735409705342, 0.010086430778857153, 0.0051407624061164305, 0.00033687136280769945, 5.4697851611291335e-05]
[317, 0.64]
[9.38285529155652, 0.0783869177713828, 0.02667957239786916, 0.0038005978436400686, 0.00021800545406197027]
[318, 0.98]
[

[136, 0.0]
[327, 0.0]
[1.8699817599338258, 0.0056726156888281965, 0.005635756556365487, 0.00016612406729663563, 9.319808990025113e-05]
[137, 0.8]
[0, 0.011358875425413946, 0.007125186767710287, 0.002249901473798517, 0.00020826757612232922]
[138, 0.3]
[0, 0.0, 0.0, 0.0, 0.0]
[139, 0.0]
[0, 0.0, 4.6017803367766923e-07, 0.0, 1.7703635894844086e-08]
[328, 0.0]
[0.42224415808906374, 0.01322233817723654, 0.004167550073339444, 0.0009433071240634041, 1.768157622152472e-05]
[261, 0.04]
[7.455054313460717, 0.04303755445054302, 0.03760950193429953, 0.0006465129738916402, 0.0003861500753740925]
[329, 1.0]
[0.8832168480105841, 0.013255746012453447, 0.005355839094130444, 9.717120280852599e-05, 4.7180840648951884e-05]
[456, 0.46]
[1.2800502718708069, 0.05634848282796708, 0.04803352403605935, 0.001154035639622431, 0.0008439305219094399]
[457, 1.0]
[0.09968133160005307, 0.03817430761597447, 0.0016106781581868687, 0.0018284143936553976, 1.3879205954397544e-05]
[17, 0.0]
[1.6581109197076223, 0.0651236414

[150, 0.0]
[0, 0.005966397921776062, 9.041129913152818e-05, 0.0003038680096611306, 4.3009451428671893e-07]
[465, 0.0]
[0.2252508848043502, 0.031204325427279515, 0.049271783717035555, 0.010744319898852245, 1.3382393156027023e-05]
[394, 1.0]
[0, 0.0008044490567444688, 1.765474382966703e-05, 0.0008044490567444688, 1.8203909269289015e-07]
[395, 0.0]
[6.306193282291587, 0.0012202171431159614, 0.009295947400020366, 1.5942927666416232e-05, 2.0215020546311162e-05]
[270, 0.52]
[0.8060768890162648, 0.011176205205873223, 0.001360302166243377, 0.00012430374347677595, 7.772420850567819e-06]
[79, 0.0]
[0.12124912785264198, 0.003970708747141929, 0.00011274739046974277, 0.00014787040235856433, 1.7034270110271727e-06]
[27, 0.0]
[0, 0.010422441052049168, 0.0019911437140306934, 0.003951149009502894, 6.563869713462504e-06]
[80, 0.0]
[0, 0.0, 2.1830459797995477e-05, 0.0, 1.3909744920006604e-07]
[81, 0.0]
[0.09815624213018816, 0.012198644354608268, 0.0006032013755615368, 0.00033217290249761266, 1.0969163250